In [14]:
import pickle
import os
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [32]:
import numpy as np
import pywt

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)
    imArray /= 255;
    # compute coefficients
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0] *= 0;

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [63]:
test_images_dir = "./testimages/"
test_X = []
test_Y = []
for img in os.scandir(test_images_dir):
    this_img = cv2.imread(img.path)
    footballer_name = os.path.basename(img.path)
    test_Y.append(os.path.splitext(footballer_name)[0][0:-1])
    
    face_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(this_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    (x,y,w,h) = faces[0]
    roi_color = this_img[y:y+h, x:x+w]
    
    cropped_img = np.array(roi_color)
    scalled_raw_img = cv2.resize(cropped_img, (32, 32))
    img_har = w2d(cropped_img,'db1',5)
    scalled_img_har = cv2.resize(img_har, (32, 32))
    combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1))).astype(float)
    combined_img = combined_img.reshape(4096,).astype(float)
    test_X.append(combined_img)
test_Y

['bernardo', 'bernardo', 'debruyne', 'debruyne', 'haaland', 'rodri']

In [64]:
from sklearn.pipeline import Pipeline

In [65]:
import joblib
pipe = joblib.load('saved_model.pkl')
result = pipe.predict(test_X)

In [95]:
keys = {0:'Bernardo', 1:'De Bruyne', 2:'Foden', 3:'Haaland', 4:'Rodri'}
correct = 0;
count = 0;
for i in result:
    print('| {:<25} | {:<20} |'.format("PREDICTION : "+keys[i], " ACTUAL : "+test_Y[count]))
    if test_Y[count] == keys[i].lower().replace(" ", ''): correct +=1 
    count += 1
print("\nAccuracy on Test Cases: ", correct/count)

| PREDICTION : Rodri        |  ACTUAL : bernardo   |
| PREDICTION : Bernardo     |  ACTUAL : bernardo   |
| PREDICTION : De Bruyne    |  ACTUAL : debruyne   |
| PREDICTION : De Bruyne    |  ACTUAL : debruyne   |
| PREDICTION : Haaland      |  ACTUAL : haaland    |
| PREDICTION : Rodri        |  ACTUAL : rodri      |

Accuracy on Test Cases:  0.8333333333333334
